# Convergence: Solow Model

### Solow Model

A representative agent uses capital $k_t$ to produce $y_t$ using the following production function:

$$y_t = k_t^{\alpha}$$

He chooses to consume an amount $c_t \in ]0, y_t]$ and invests what remains:

$$i_t = y_t - c_t$$ 

He accumulates capital $k_t$ according to:

$$k_{t+1} = \left( 1-\delta \right) k_{t} + i_{t}$$

where $\delta$ is the depreciation rate and $i_t$ is the amount invested.

The goal of the representative agent is to maximize:

$$\sum_{t\geq 0} \beta^t U(c_t)$$

where $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ and $\beta<1$ is the discount factor.

For now, we ignore the objective and assume that the saving rate $s=\frac{c_t}{y_t}$ is constant over time.

__Create a `NamedTuple` to hold parameter values $\beta=0.96$, $\delta=0.1$, $\alpha=0.3$, $\gamma=4$.__ 

In [1]:
p = (;β=0.96, δ=0.1, α=0.3, γ=4.0)

(β = 0.96, δ = 0.1, α = 0.3, γ = 4.0)

__Write down the formula of function $f$ such that $k_{t+1}$: $k_{t+1} = f(k_t)$.__

$$k_{t+1}= (1-\delta) k_t + (1-s) k_t^{\alpha}$$

__Define a function `f(k::Float64, p::NamedTuple)::Float64` to represent $f$ for a given calibration__

In [7]:
# function f(k::Float64, p::NamedTuple)

# we added a keyword argument to specify the saving rate
function f(k, p; s=0.2)

    (;α, δ) = p # keyword unpacking syntax

    val = k*(1-δ) + s*k^α

    return val 
end

f (generic function with 2 methods)

In [6]:
f(0.1, p)

0.1902374467254545

__Write a function `simulate(k0::Float64, T::Int, p::NamedTuple)::Vector{Float64}` to compute the simulation over `T` periods starting from initial capital level `k0`.__

In [10]:
function simulate(k0, T, p; s=0.5)

    sim = [k0]

    for i ∈ 1:T    # same as for i in ... or for i=...
        # in Julia, intervals contain the lower and upper bound
        k1 = f(k0, p; s=s)

        # add new value to simulation vector
        push!(sim, k1)

        k0 = k1
    end

    return sim
end

simulate (generic function with 1 method)

In [11]:
sim = simulate(0.5, 100, p;)

101-element Vector{Float64}:
 0.2
 0.30340677254400195
 0.4129080792968781
 0.525003373019628
 0.6373491155565568
 0.7483344417178636
 0.856841626057419
 0.9620988898971137
 1.0635841029776287
 1.1609587708409257
 ⋮
 2.6876186883735818
 2.6879113388853018
 2.6881835130861362
 2.6884366430712516
 2.6886720608576273
 2.688891005366757
 2.689094628921639
 2.6892840032916374
 2.6894601253165105

In [13]:
# sometimes, we want to avoid using too much memory
# in that case we try to do as many inplace operations as possible

function simulate_preallocate(k, T, p; s=0.2)
    v = zeros(T) # allocates memore
    v[1] = k
    for t = 1:(T-1)
        k0 = v[t]
        v[t+1] = f(k0, p; s=s)
    end
    return v
end

simulate_preallocate (generic function with 1 method)

In [18]:
@time simulate(0.2, 1000000, p);
@time simulate_preallocate(0.2, 1000000, p);

  0.067737 seconds (14 allocations: 9.781 MiB)
  0.055613 seconds (2 allocations: 7.629 MiB)


__Make a nice plot to illustrate the convergence. Do we get convergence from any initial level of capital?__

In [ ]:
pl= plot(simulate(0.5, 100, p;); label="baseline", title="Convergence of Solow Model")
plot!(pl, simulate(6.0, 100, p;); label="high initial capital")
plot!(pl, simulate(0.5, 100, p;s=0.2); label="lower saving rate")

__Suppose you were interested in using `f` to compute the steady-state. What would you propose to measure convergence speed? To speed-up convergence? Implement these ideas.__

In [20]:
function steady_state(p; s=0.2)
    sim = simulate(0.1, 1000, p)
    return sim[end]
end

steady_state (generic function with 1 method)

In [21]:
steady_state(p)

2.691800385264708